<a href="https://colab.research.google.com/github/yalopez84/Interpretable_method/blob/master/Training_sbrt_test_mnr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install sentence_transformers
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.system("export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:20")

In [ ]:
import datasets

snli = datasets.load_dataset('snli', split='train')
mnli = datasets.load_dataset('glue', 'mnli', split='train')

mnli=mnli.remove_columns('idx')
snli = snli.cast(mnli.features)
dataset = datasets.concatenate_datasets([snli, mnli])
del snli, mnli
dataset = dataset.filter(
    lambda x: True if x['label'] == 0 else False
)

In [ ]:
from sentence_transformers import InputExample
from tqdm.auto import tqdm  
train_samples = []
for row in tqdm(dataset):
    train_samples.append(InputExample(
        texts=[row['preimse'], row['hypothesis']]
    ))

In [ ]:
from sentence_transformers import datasets
batch_size = 16
loader = datasets.NoDuplicatesDataLoader(
    train_samples, batch_size=batch_size)

In [ ]:
from sentence_transformers import models, SentenceTransformer

bert = models.Transformer('bert-base-uncased')
pooler = models.Pooling(
    bert.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True
)

model = SentenceTransformer(modules=[bert, pooler])

In [ ]:
from sentence_transformers import losses
loss = losses.MultipleNegativesRankingLoss(model)

In [ ]:
epochs = 1
warmup_steps = int(len(loader) * epochs * 0.1)

model.fit(
    train_objectives=[(loader, loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    output_path='/content/drive/MyDrive/Almacen_phd/FB13/sbert_test_mnr2',
    show_progress_bar=True
)  